# Shot Boundary Detection in Cricket Sample Videos using Bayesian Inference

Here we try to model the Shot Boundary Detection problem using Bayesian Inference approach. The sample videos of cricket highlights have been used. In the dataset we have 26 videos total, which have been manually labeled for shot boundaries (only Cuts). Each video is of approx. 3-5 mins having 3k-6k frames. 

The steps involved will be:
1. It is a binary classification problem, where features X are taken as the summed up histogram difference values of consecutive frames.
2. C = {Cuts(c), Non-Cuts(c')}, X = {sum(hist-diff(Icurr, Iprev))}
3. P(C | X) = P(X | C) P(C) / (P(X|c).P(c) + P(X|c').P(c'))
4. Define the training, validation and test sets. (60% (16 videos), 20% (5 videos), 20% (5 videos))
5. Extract the #+ve sample values, and #-ve sample values, for all the training videos. Save in file.
6. 

In [3]:
import cv2
import numpy as np
import os

DATASET = "/opt/datasets/cricket/ICC_WT20"
LABELS = "/opt/datasets/cricket/gt_ICC_WT20"

In [14]:
# Split the dataset files into training, validation and test sets
def split_dataset_files():
    filenames = sorted(os.listdir(DATASET))         # read the filenames
    filenames = [t.split('.')[0] for t in filenames]   # remove the extension
    return filenames[:16], filenames[16:21], filenames[21:]

In [20]:
# function to extract the features from a list videos
# list of videos for which hist_diff values are to be extracted
def extract_hist_diff_vids(vids_lst):
    # iterate over the videos to extract the hist_diff values
    for idx, vid in enumerate(vids_lst):
        get_hist_diff(os.path.join(DATASET, vid+'.avi'))

In [26]:
# read the video file and extract the hist_diff values in a list and return the list
def get_hist_diff(vid_path):
    cap = cv2.VideoCapture(vid_path)
    if not cap.isOpened():
        import sys
        print "Capture object not opened"
        sys.exit()
    print "Opened Video : "+vid_path
    
    ret, frame = cap.read()
    while(cap.isOpened()):
        ret, frame = cap.read()
    cap.release()

In [ ]:
# function to get the weighted chi squared values for diff-hist
def weighted_chi_sq_test(srcVideoPath, destPath, th):
    # get the VideoCapture object
    cap = cv2.VideoCapture(srcVideoPath)
    
    # if the videoCapture object is not opened then exit without traceback
    if not cap.isOpened():
        import sys
        print("Error reading the video file !!")
        sys.exit(0)
    
    dimensions = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    fps = cap.get(cv2.CAP_PROP_FPS)
    #fourcc = cv2.cv.CV_FOURCC(*'XVID')
    
    N = 256     # no of bins for histogram
    frameCount = 0
    color = ('b','g','r')     # defined for 3 channels
    # uncomment following line for grayscale
    #color = ('b')
    prev_hist = np.zeros((256, 3))      # histograms for 3 channels
    curr_hist = np.zeros((256, 3))
    diffs = np.zeros((1, 1))        # single number appended to vector
    while(cap.isOpened()):
        # Capture frame by frame
        ret, frame = cap.read()
        # print(ret)
    
        if ret==True:
            # frame = cv2.flip(frame)
            frameCount = frameCount + 1
            
            for i,col in enumerate(color):
                # cv2.calcHist(images, channels, mask, histSize, ranges[, hist[, accumulate]])
                curr_hist[:,i] = np.reshape(cv2.calcHist([frame], [i], None, [N], [0,N]), (N,))
            
            if frameCount > 1:
                # find the L1 distance of the current frame hist to previous frame hist 
                #dist = np.sum(abs(curr_hist - prev_hist), axis=0)
                ######################################################
                # find the squared distances
                sq_diffs = (curr_hist - prev_hist)**2
                # max values of corresponding bins max(Hi(k), Hj(k))
                max_bin_values = np.maximum(curr_hist, prev_hist)
                # find sq_diffs/max_bin_values, will generate nan for 0/0 cases
                d1 = np.divide(sq_diffs, max_bin_values)
                # convert nan to 0
                nan_indices = np.isnan(d1)
                d1[nan_indices] = 0
                # Identify which channel is RGB
                # multiply col 0 by gamma (Blue), col 1 by beta (green)
                # and col 2 by alpha (red)
                gamma_beta_alpha = np.array([0.114, 0.587, 0.299])
                # multiply each column by const
                d1 = d1*gamma_beta_alpha
                #d1 = np.sum(np.sum(d1, axis=1))     # sum the columns (3 channels)
                #sum up everything
                dist = np.sum(d1)/(3.0*N)       # N = 256 bins
                ######################################################
                #diffs.append(dist)
                diffs = np.vstack([diffs, dist])
                print("dist appended = ", type(dist), dist)
                print("diffs shape = ", type(diffs), diffs.shape)
        
            np.copyto(prev_hist, curr_hist)        
            
            ### write the flipped frame
            cv2.imshow('frame', frame)
            # Our operations on the frame come here
            # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            # Display the resulting frame
            # cv2.imshow('frame', gray)
            if cv2.waitKey(10) == 27:
                print('Esc pressed')
                break
            
        else:
            break

    # When everything done, release the capture
    cap.release()
    #out.release()
    cv2.destroyAllWindows()
    plt.close("all")
    # draw the line graph of the diffs signal
    for i,col in enumerate(color):
        fig = plt.figure(i)
        plt.title("Weighted Chi-Test of Consecutive frame Histograms")
        plt.xlabel("Frame No.")
        plt.ylabel("Wtd. Chi-Sq Dist. of H(fi) and H(fj)")
        ax = fig.add_subplot(111)
        plot_points = diffs[:,i]
        plt.plot(plot_points, color=col, label=str(i))
        # loop over only the peaks above threshold
        peaks = [(x,y) for (x,y) in zip(range(len(plot_points)),plot_points) if y>th]
        for ind,(x,y) in enumerate(peaks):
            ax.annotate('(%s,s'%x+str(ind)+')', xy=(x,y), textcoords='data')
        plt.grid()
        #plt.plot(diffs)
        plt.savefig(os.path.join(destPath,"hist_"+str(col)+".png"), bbox_inches='tight')
    # save diff_hist to disk    
    outfile = file(os.path.join(destPath,"diff_hist.bin"), "wb")
    np.save(outfile, diffs)
    outfile.close()
    return diffs

In [ ]:
# function to learn the model

In [17]:
#f = sorted(os.listdir(DATASET))
#gt = sorted(os.listdir(LABELS))
#f1 = [t.split('.')[0] for t in f]
#print f1
#for i,name in enumerate(f):
#    print(i, name.split('.')[0], gt[i])

In [24]:
if __name__=="__main__":
    # Divide the samples files into training set, validation and test sets
    train_lst, val_lst, test_lst = split_dataset_files()
    #print(train_lst, len(train_lst))
    #print 100*"-"
    #print val_lst
    #print 100*"-"
    #print test_lst
    
    # Extract the histogram difference features from the training set
    extract_hist_diff_vids(test_lst)

Opened Video : /opt/datasets/cricket/ICC_WT20/ICC WT20 Scotland vs Zimbabwe Highlights.avi
Opened Video : /opt/datasets/cricket/ICC_WT20/ICC WT20 South Africa v Sri Lanka - Match Highlights.avi
Opened Video : /opt/datasets/cricket/ICC_WT20/ICC WT20 South Africa vs West Indies - Match Highlights.avi
Opened Video : /opt/datasets/cricket/ICC_WT20/ICC WT20 Sri Lanka v Afghanistan Cricket Match Highlights.avi
Opened Video : /opt/datasets/cricket/ICC_WT20/ICC WT20 West Indies v India - Semi-Final Highlights.avi
